In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
from tensorflow.keras.models import load_model
from sklearn.model_selection import train_test_split
df=pd.read_excel('MOOD TRAK.xlsx')
df.head()

,Time Stamp,Day,Activities,Start,End,Quality,Note
0,2024-05-01 07:00:00,Wednesday,sleep,23:00:00,05:00:00,okay,Bangun pagi untuk sholat subuh.
1,2024-05-01 07:00:00,Wednesday,selfcare,05:00:00,05:30:00,good,"Mandi dan bersiap, air dingin di pagi hari tid..."
2,2024-05-01 07:00:00,Wednesday,eating,05:30:00,06:00:00,good,Sarapan pagi dengan roti bakar dan susu.
3,2024-05-01 07:00:00,Wednesday,workout,06:00:00,07:00:00,excellent,Jogging sekitar kos-kosan. Bertemu banyak anak...
4,2024-05-01 07:00:00,Wednesday,traveling,07:00:00,07:30:00,excellent,"Berangkat ke kampus, perjalanan sekitar 2 km d..."


In [3]:
# Load the trained model
model = load_model('quality_condition_model.h5')

In [4]:
# Process the data
df_new = df.drop(columns=['Note'])
df_new['Time Stamp'] = pd.to_datetime(df_new['Time Stamp']).dt.date
quality_mapping = {'terrible': 1, 'bad': 2, 'okay': 3, 'good': 4, 'excellent': 5}
df_new['Quality'] = df_new['Quality'].map(quality_mapping)
df_new['Start'] = pd.to_datetime(df_new['Start'], format='%H:%M:%S').dt.time
df_new['End'] = pd.to_datetime(df_new['End'], format='%H:%M:%S').dt.time
df_new['Duration'] = pd.to_timedelta(df_new['End'].astype(str)) - pd.to_timedelta(df_new['Start'].astype(str))
df_new['Duration'] = df_new['Duration'].apply(lambda x: x + pd.Timedelta(days=1) if x.days < 0 else x)
df_new['Duration'] = df_new['Duration'].apply(lambda x: int(pd.to_timedelta(x).total_seconds() // 60))
df_new['Week'] = pd.to_datetime(df_new['Time Stamp']).dt.isocalendar().week
df_new['WeekDay'] = df_new['Week'].astype(str) + '-' + df_new['Day']

df_new

,Time Stamp,Day,Activities,Start,End,Quality,Duration,Week,WeekDay
0,2024-05-01,Wednesday,sleep,23:00:00,05:00:00,3,360,18,18-Wednesday
1,2024-05-01,Wednesday,selfcare,05:00:00,05:30:00,4,30,18,18-Wednesday
2,2024-05-01,Wednesday,eating,05:30:00,06:00:00,4,30,18,18-Wednesday
3,2024-05-01,Wednesday,workout,06:00:00,07:00:00,5,60,18,18-Wednesday
4,2024-05-01,Wednesday,traveling,07:00:00,07:30:00,5,30,18,18-Wednesday
...,...,...,...,...,...,...,...,...,...
1215,2024-08-06,Tuesday,entertainment,17:45:00,19:00:00,3,75,32,32-Tuesday
1216,2024-08-06,Tuesday,eating,19:00:00,19:30:00,3,30,32,32-Tuesday
1217,2024-08-06,Tuesday,study,19:30:00,21:00:00,3,90,32,32-Tuesday
1218,2024-08-06,Tuesday,dating,21:00:00,23:00:00,3,120,32,32-Tuesday


In [5]:
# Define weights for each activity (higher weight means higher significance)
activity_weights = {
    'sleep': 0.2,
    'study': 0.1,
    'work': 0.35,
    'dating':0.05,
    'selfcare':0.05,
    'traveling':0.05,
    'entertainment':0.1,
    'eating':0.05,
    'workout':0.05
    # Add other activities and their weights here
}

# Define classification thresholds based on the overall day quality score
classification_thresholds = {
    'terrible': 0.116667,
    'bad': 0.183333,
    'okay': 0.292308,
    'good': 0.540000,
    'excellent': 0.775,
}
# Calculate overall day quality score for each day
df_new['Overall_Quality'] = df_new['Quality'] * df_new['Activities'].map(activity_weights)
daily_quality = df_new.groupby(['WeekDay']).agg({'Overall_Quality': 'mean'}).reset_index()
daily_quality

,WeekDay,Overall_Quality
0,18-Friday,0.319231
1,18-Saturday,0.230000
2,18-Sunday,0.282143
3,18-Thursday,0.226923
4,18-Wednesday,0.257143
...,...,...
93,31-Thursday,0.346154
94,31-Tuesday,0.519231
95,31-Wednesday,0.364286
96,32-Monday,0.346154


In [6]:
X_new = daily_quality.drop(columns=['WeekDay'])

In [7]:

# Make predictions
predictions = model.predict(X_new)

# Add the predicted classes to the new dataset
daily_quality['Predicted_Day_Condition'] = predictions.argmax(axis=1)

# Print or save the new dataset with predictions
daily_quality

4/4 [==============================] - 0s 5ms/step


,WeekDay,Overall_Quality,Predicted_Day_Condition
0,18-Friday,0.319231,2
1,18-Saturday,0.230000,3
2,18-Sunday,0.282143,3
3,18-Thursday,0.226923,3
4,18-Wednesday,0.257143,3
...,...,...,...
93,31-Thursday,0.346154,2
94,31-Tuesday,0.519231,2
95,31-Wednesday,0.364286,2
96,32-Monday,0.346154,2


In [8]:
# Create a new LabelEncoder instance
label_encoder = LabelEncoder()
original_class_labels = ['bad', 'excellent', 'good', 'okay','terrible']
# Fit the LabelEncoder to the original class labels
label_encoder.fit(original_class_labels)

# Inverse transform the predicted classes to get the original class labels
predicted_day_condition_labels = label_encoder.inverse_transform(daily_quality['Predicted_Day_Condition'])

# Add the predicted class labels to the new dataset
daily_quality['Predicted_Day_Condition_Labels'] = predicted_day_condition_labels

# Print or save the new dataset with predicted class labels
daily_quality

,WeekDay,Overall_Quality,Predicted_Day_Condition,Predicted_Day_Condition_Labels
0,18-Friday,0.319231,2,good
1,18-Saturday,0.230000,3,okay
2,18-Sunday,0.282143,3,okay
3,18-Thursday,0.226923,3,okay
4,18-Wednesday,0.257143,3,okay
...,...,...,...,...
93,31-Thursday,0.346154,2,good
94,31-Tuesday,0.519231,2,good
95,31-Wednesday,0.364286,2,good
96,32-Monday,0.346154,2,good


In [9]:
# Classify each day based on the overall quality score
def classify_day(quality_score):
    for category, threshold in classification_thresholds.items():
        if quality_score < threshold:
            return category
    return 'excellent'

daily_quality['Real Category'] = daily_quality['Overall_Quality'].apply(classify_day)

daily_quality

,WeekDay,Overall_Quality,Predicted_Day_Condition,Predicted_Day_Condition_Labels,Real Category
0,18-Friday,0.319231,2,good,good
1,18-Saturday,0.230000,3,okay,okay
2,18-Sunday,0.282143,3,okay,okay
3,18-Thursday,0.226923,3,okay,okay
4,18-Wednesday,0.257143,3,okay,okay
...,...,...,...,...,...
93,31-Thursday,0.346154,2,good,good
94,31-Tuesday,0.519231,2,good,good
95,31-Wednesday,0.364286,2,good,good
96,32-Monday,0.346154,2,good,good


In [10]:
# Compare the predicted and real categories
daily_quality['Comparison'] = daily_quality['Predicted_Day_Condition_Labels'] == daily_quality['Real Category']
# Summary of comparison
correct_predictions = daily_quality['Comparison'].sum()
total_predictions = len(daily_quality)
accuracy = correct_predictions / total_predictions

print(f"Correct Predictions: {correct_predictions}/{total_predictions}")
print(f"Accuracy: {accuracy:.2%}")

Correct Predictions: 98/98
Accuracy: 100.00%
